In [29]:
import os
import glob
import pandas as pd
import json
from collections import defaultdict
import xml.etree.ElementTree as et
from BeautifulSoup import BeautifulSoup as soup
import time
import matplotlib.pyplot as plt
import csv


Data Collection (Creating Dataframe)

In [2]:
#directory where data lives
data_dirs = glob.glob("/Users/bxie/Documents/ai2_projects/*")

columns = ['id', 'project_name', 'user_name', 'num_screens']

NUM_PROJECTS = len(data_dirs) #Number of projects I'm parsing

"Number of Projects: " + str(NUM_PROJECTS)

'Number of Projects: 5228'

In [57]:
df = pd.DataFrame.from_csv("omit/ai2_data.csv")
df = df.fillna(0)

In [4]:
#Functions to parse files

#Given project.properties file contents (as string),
#return dictionary with relevant info
def parse_properties(inp):
    lines = inp.split("\n") #split by line
    names = lines[0].split(".") #split by dot
    
    output = {"user_name": names[1][3:],
            "project_name": names[2].lower().strip()}

    return output

#Given scm file (as string), returns dictionary
#with keys as components and values as frequency of components
def parse_scm(inp):
    output = defaultdict(int)
    lines = inp.split("\n")
    json_str = lines[2] #json is all here; rest of lines don't matter
    j = json.loads(json_str)
    
    if "$Components" in j['Properties']:
        for component in j['Properties']['$Components']: #accessing list of components
            output['c_'+component['$Type']] += 1 #adding that component to dictionary
    return dict(output)

#Given bky file (as string), returns dictionary
#with keys as block types and values as frequency
def parse_bky(inp):
    output = defaultdict(int)
    b = soup(inp) #parsing xml w/ beautifulsoup
    blocks = b.findAll("block")
    for blk in blocks:
        output['b_'+blk["type"]]+=1
#     print dict(output)
    return dict(output)

#Given list of frequency dictionaries, 
#merge dictionaries (summing similar keys) and returning it
def merge_dicts(dicts_list):
#     if len(dicts_list)>1:
#         print 'MULTIPLE SCREENS!' #make sure this thing is working...
    result = {}
    keymap = {}
    for mydict in dicts_list:
        for (k,v) in mydict.items():
            if not keymap.has_key(k):
                keymap[k] = []
            keymap[k].append(v)
    for (k,v) in keymap.items():
        result[k] = sum(int(x) for x in keymap[k])
    return result    


In [49]:
#Feature extraction
#THIS TAKES TIME (~10 min)

df = pd.DataFrame(columns=columns)
t0 = time.time()

#Figure out how many missing files
num_missing_prop = 0
num_missing_scm = 0
num_missing_bky = 0

#Number of projects to process
num_projects = 100

for my_dir in data_dirs:          
# for x in range(0,num_projects):
#     my_dir = data_dirs[x]
    values = {} #dictionary of things to add to dataframe. key corresponds to column in df
    prop = {}
    scm = {}
    bky = {}
    
    values['id'] = str.split(my_dir, '/')[-1] #getting id of project from directory
    
    #properties
    dir_prop = glob.glob(my_dir+"/project.properties")
    if(len(dir_prop)>0):
        file_prop = open(dir_prop[0], 'r')
        prop = parse_properties(file_prop.read())
    else:
        num_missing_prop+=1
    
    #scm
    dir_scm = glob.glob(my_dir+"/*.scm")
    if(len(dir_scm)>0):
        scm_dicts = []
        #Getting dictionary of info from each screen
        for d in dir_scm:
            file_scm = open(d, 'r')    
            scm_dicts.append(parse_scm(file_scm.read()))
            
        #merging dictionaries of screen info
        if(len(dir_scm)>1):
            scm = merge_dicts(scm_dicts)
        else:
            scm = scm_dicts[0]
        
        values['num_screens'] = len(scm_dicts)
    else:
        num_missing_scm+=1

    #bky
    dir_bky = glob.glob(my_dir+"/*.bky") 
    if(len(dir_bky)>0):
        #equivalent logic as scm. too lazy to write a function...
        bky_dicts = []
        for d in dir_bky:
            file_bky = open(d, 'r')    
            bky_dicts.append(parse_bky(file_bky.read()))        
        
        if(len(dir_bky)>1):
            bky = merge_dicts(bky_dicts)
        else:
            bky = bky_dicts[0]
    else:
        num_missing_bky+=1
        
    stats = {'num_components':sum(scm.values()),
             'num_components_unique': len(scm),
             'num_blocks':sum(bky.values()),
             'num_blocks_unique':len(bky)}
    values = dict(values.items() + stats.items() + prop.items() + scm.items() + bky.items())
    
    df = df.append(values, ignore_index=True)

t1 = time.time()
delta = t1 - t0

print "Execution Time: " + str(delta) + "sec (" + str(delta/60) + " min)" #around 8 min
print
print "Num of missing properties files: "+str(num_missing_prop)
print "Num of missing screen (scm) files: "+str(num_missing_scm)
print "Num of missing block (bky) files: "+str(num_missing_bky)
print
num_no_blks = len(df[df.num_blocks<2])
print "Number of projects w/ <2 blocks: " + str(num_no_blks) + " ("+str(num_no_blks*100/NUM_PROJECTS)+" % of projects)"

num_no_comp = len(df[df.num_components==0])
print "Number of projects w/ no components: " + str(num_no_comp) + " ("+str(num_no_comp*100/NUM_PROJECTS)+" % of projects)"


Execution Time: 575.171663046sec (9.5861943841 min)



Data Aggregation (Manipulating Data)

- Grouping similar blocks and components

In [38]:
#Functions for data manipulation

#Given dataframe, return dataframe w/ new column col_name that sums given features
def add_agg_column(df_inp, col_name, features):
    df = df_inp.copy()
    df[col_name] = 0 #defining column
    index = 0;
    for row in df[features].iterrows(): #iterating through rows
        val = int(sum(row[1].values))
        df.loc[index, col_name] = val
        index = index+1
    return df

# given path to 2 column csv, return dictionary where key is first col
# key_as_first: true if first col is key (false is 2nd is)
# 1st col: component/block name
# 2nd col: component/block category
def csv_to_dict(path, key_as_first=True):
    reader = csv.reader(open(path, 'r'))
    d = {}
    for row in reader:
        if key_as_first:
            k, v = row
            d[k] = v
        else:
            v, k = row
            if k in d:
                d[k]+=[v] #add to list
            else:
                d[k]=[v]
    return d

In [56]:
sensors = ['c_AccelerometerSensor', 'c_LocationSensor', 'c_ProximitySensor', 'c_Clock', 'c_BarcodeScanner', 'c_NearField', 'c_ProximitySensor']
df_test = add_agg_column(df, 'num_sensor_components', sensors)
df_test[['num_sensor_components']].head(10)

,num_sensor_components
0,1
1,0
2,0
3,1
4,0
5,0
6,0
7,2
8,0
9,1


In [49]:
df[['num_blocks_unique', 'num_components_unique', 'num_components']].describe()

,num_blocks_unique,num_components_unique,num_components
count,5228.000000,5228.000000,5228.000000
mean,7.129304,3.284621,6.047819
std,7.060597,2.300568,11.711114
min,0.000000,0.000000,0.000000
25%,2.000000,2.000000,2.000000
50%,5.000000,3.000000,4.000000
75%,11.000000,4.000000,6.000000
max,56.000000,41.000000,350.000000


In [55]:
#Get list of all column names pertaining to blocks or components
blocks = [name for name in df.columns if name[:2]=="b_"] #111 total
components = [name for name in df.columns if name[:2]=="c_"] #57 total
category_comp = csv_to_dict("component_category.csv", False)


blocks_no_prefix=[] #blocks without "b_" in front
section = [] #category block lives in ("list, math, etc.")
for txt in blocks:
    text = txt[2:]
    sect = text.split("_")[0]
    section.append(sect)
#     print sect
    blocks_no_prefix.append(txt[2:])

#key: block (w/ "b_" prefix). value: category it maps to
#THERE HAS GOT TO BE A BETTER WAY TO DO THIS (put in spreadsheet, convert to csv, import as df?)

# print category_comp['ui']

# print len(blocks)
# print sum(df.num_blocks) / 5228
# df[['num_blocks']].describe()

print category_comp

{'layout': ['c_HorizontalArrangement', 'c_TableArrangement', 'c_VerticalArrangement'], 'lego': ['c_NxtColorSensor', 'c_NxtLightSensor', 'c_NxtSoundSensor', 'c_NxtDirectCommands', 'c_NxtDrive', 'c_NxtTouchSensor', 'c_NxtUltrasonicSensor'], 'media': ['c_Camera', 'c_Sound', 'c_TextToSpeech', 'c_Player', 'c_SoundRecorder', 'c_SpeechRecognizer', 'c_ImagePicker', 'c_Camcorder', 'c_VideoPlayer', 'c_YandexTranslate'], 'storage': ['c_TinyWebDB', 'c_TinyDB', 'c_FusiontablesControl', 'c_File'], 'ui': ['c_Button', 'c_Label', 'c_Slider', 'c_Notifier', 'c_TextBox', 'c_Image', 'c_ListPicker', 'c_ListView', 'c_PasswordTextBox', 'c_WebViewer', 'c_CheckBox', 'c_Spinner', 'c_DatePicker', 'c_TimePicker'], 'connect': ['c_ActivityStarter', 'c_BluetoothClient', 'c_Web', 'c_BluetoothServer'], 'social': ['c_PhoneCall', 'c_Texting', 'c_PhoneNumberPicker', 'c_Sharing', 'c_ContactPicker', 'c_Twitter', 'c_EmailPicker'], 'sensor': ['c_Clock', 'c_AccelerometerSensor', 'c_LocationSensor', 'c_OrientationSensor', 'c_Ne

In [16]:
print len(blocks)

111


In [54]:
#Looking at # of projects that use variables
print len(df[df.b_global_declaration>0]) #/float(NUM_PROJECTS)
print
print len(df[df.b_lexical_variable_set>0]) #/float(NUM_PROJECTS)
print 
print len(df[df.b_lexical_variable_get>0]) #/float(NUM_PROJECTS)
print 


1514

1359

2219



In [53]:
#Looking at number of projects that use procedures
num_proc_def = len(df[df.b_procedures_defnoreturn>0]) + len(df[df.b_procedures_defreturn>0]) #projects that def procedures
num_proc_call = len(df[df.b_procedures_callreturn>0]) + len(df[df.b_procedures_callnoreturn>0]) #projects that call procedures

print num_proc_def 
print float(float(num_proc_def) / NUM_PROJECTS)
print
print num_proc_call
print float(float(num_proc_call) / NUM_PROJECTS)

845
0.161629686305

781
0.149387911247


In [57]:
#Get dataframe with most populated columns (to identify most relevant features)
count = df.count()
count.sort(ascending=False) #series of column names, sorted by most populated columns
top_features = count.keys()[:30] 
df_top = df[top_features]
# top_features
# df_top.head(10)

In [58]:
#Removing projects that are "certainly" incomplete
df_work = df_top[df_top.num_blocks>=2][df_top.num_components>0]

#Looking at projects that "might" work (removing "certainly incomplete" projects)
# df_work.describe()

/Users/bxie/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:1808: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [59]:
#Dump data to csv file
df.to_csv("omit/ai2_data.csv")
df_top.to_csv("omit/ai2_data_top.csv")
df_work.to_csv("omit/ai2_data_work.csv")

print "Done."

Done.


In [9]:
# Finding tutorials by name
tutorials = {"hello_purr":"purr",
             "talk_to_me":"talk",
             "ball_bounce":"bounce",
             "digital_doodle:" : "doodle",
             "magic_8_ball": "magic",
             "paintpot" : "paint",
             "i_have_dream" : "dream",
             "pic_call": "piccall",
             "video_wall": "videowall|video_wall",
             "stockquotes": "stockquotes|stock_quotes",
             "mini_golf" : "minigolf|mini_golf",
             "space_invaders" : "spaceinvader|space_invader",
             "no_text":"notext|driving",
             "mash": "mash"}

tutorials_count = {}
total_tutorials = 0
total_working_tutorials = 0

for entry in tutorials:
    search_term = tutorials[entry]
    count_work = len(df_work[df_work.project_name.str.contains(search_term)])
    count_all = len(df[df.project_name.str.contains(search_term)])
    tutorials_count[entry] = count_work, count_all

    total_working_tutorials += count_work
    total_tutorials += count_all

print tutorials_count
print "total tutorials: " + str(total_tutorials) + "(" + str(float(total_tutorials)/ NUM_PROJECTS*100) +"%)"
print "total working tutorials: " + str(total_working_tutorials) + "(" + str(float(total_working_tutorials)/ NUM_PROJECTS*100) +"%)"

{'ball_bounce': (109, 112), 'paintpot': (162, 185), 'i_have_dream': (37, 42), 'digital_doodle:': (67, 71), 'video_wall': (3, 3), 'pic_call': (4, 4), 'magic_8_ball': (61, 68), 'mash': (122, 144), 'talk_to_me': (133, 152), 'space_invaders': (29, 30), 'mini_golf': (20, 21), 'stockquotes': (9, 9), 'hello_purr': (66, 82), 'no_text': (12, 14)}
total tutorials: 937(17.922723795%)
total working tutorials: 834(15.9525631217%)


In [10]:
#Extract relevant features to clasify tutorials

relevant_features = {}

# Given a search term, takes projects that contain the search term in project_name,
# returns dataframe of features that exist in at least threshold (float, [0,1]) of the projects
def feature_finder(search_term):
    #Ignoring meta data
    omit = ['id', 'user_name', 'project_name'] #num_blocks, num_components, num_screens may go here
    tutorial = df[df.project_name.str.contains(search_term)]
    tutorial = tutorial.dropna(axis=1, how="all")
    num_tutorials = len(tutorial)
    threshold = 0.4
    
    count = tutorial.count()
    count = dict(count)
    
    #finds features that exist in thershold of the projects
    features = [ x[0] for x in count.items() if x[1]>(num_tutorials*threshold) ]
    
    #removing meta data
    for val in omit:
        features.remove(val)
    
    # TODO: DON'T USE MEDIAN HERE. PASS IN RANGE BETWEEN QUARTILE 1-3???
    return df[features]

#Given iterable where key is feature and value is freq,
#return filtered dataframe where feature == value for all in features
def find_projects(features):
    q = ""
    for f in features.keys():
        count = features[f]
        q = q+"(" + f + ">" + str(count-1) + " or " + f + "<" + str(count+1)+") and "
    q = q[:-5] #removing trailing " and "
    print q
    return df.query(q)

#DOESN'T QUITE WORK. JUST USE SCIKIT LEARN CLASSIFICATION!
#query dataframe for tutorial given list(?) of features
def find_projects(features):
    q = ""
    for f in features.keys():
        count = int(features[f])
        q = q+"(" + f + ">" + str(count-1) + " or " + f + "<" + str(count+1)+") and "
    q = q[:-5] #removing trailing " and "
    print q
    return df.query(q)

In [11]:
all_features = [] #All features found in at least 40% of a tutorial
for tut in tutorials:
    search_term = tutorials[tut]
    features = feature_finder(search_term)
    for col in features.columns:
        if col not in all_features:
            all_features.append(col)
#     relevant_features[tut] = features.median()

feature_count = df[all_features].count()
feature_count.sort(ascending=False)
feature_count
#Find all hellopurrs given features. DOESN'T WORK!!!
# purr_features = relevant_features["hello_purr"]
# df_f = find_projects(purr_features)

# purr_features = df[all_features].median()
# find_projects(purr_features)

num_blocks                     5228
num_components                 5228
num_screens                    5228
b_component_event              4107
b_component_set_get            3313
b_component_method             2993
c_Button                       2696
b_text                         2477
b_lexical_variable_get         2219
b_math_number                  2072
c_Label                        1947
c_HorizontalArrangement        1901
b_global_declaration           1514
b_controls_if                  1478
c_Canvas                       1442
b_lexical_variable_set         1359
b_math_add                     1047
b_text_join                     932
c_Clock                         920
b_logic_boolean                 912
c_Sound                         880
b_math_compare                  870
b_math_subtract                 755
c_TextBox                       745
b_procedures_defnoreturn        727
b_lists_create_with             708
c_Image                         693
b_procedures_callnoreturn   

In [306]:
#Trying to get breakdown of tutorials for pie chart (or equiv)...kinda works, kinda doesnt. what ev.
df_tut = pd.DataFrame(columns=['name', 'might_work', 'total'])
for v in tutorials_count:
    may_work = tutorials_count[v][0]
    total = tutorials_count[v][1]
    temp = {'name': v,
            'might_work': may_work,
            'total': total}
    df_tut = df_tut.append(temp, ignore_index=True)

In [307]:
df_tut

,name,might_work,total
0,ball_bounce,109,112
1,paintpot,162,185
2,i_have_dream,37,42
3,digital_doodle:,67,71
4,video_wall,3,3
5,pic_call,4,4
6,magic_8_ball,61,68
7,mash,122,144
8,talk_to_me,133,152
9,space_invaders,29,30


In [29]:
#Looking at most populated columns
print df_top.count()
print
print df_top.count()/NUM_PROJECTS

id                          5228
project_name                5228
user_name                   5228
num_screens                 5228
num_components              5228
num_blocks                  5228
b_component_event           4107
b_component_set_get         3313
b_component_method          2993
c_Button                    2696
b_text                      2477
b_lexical_variable_get      2219
b_math_number               2072
c_Label                     1947
c_HorizontalArrangement     1901
b_global_declaration        1514
b_controls_if               1478
c_Canvas                    1442
b_lexical_variable_set      1359
b_math_add                  1047
b_text_join                  932
c_Clock                      920
b_logic_boolean              912
c_Sound                      880
b_math_compare               870
b_math_subtract              755
c_TextBox                    745
b_procedures_defnoreturn     727
b_lists_create_with          708
c_Image                      693
dtype: int

In [84]:
#acceleratometer, barcode, clock, location, nearfield, orientation, proximity

#Given dataframe, return dataframe w/ new column col_name that sums given features
def add_agg_column(df_inp, col_name, features):
    df = df_inp.copy()
    df[col_name] = 0 #defining column
    index = 0;
    for row in df[features].iterrows(): #iterating through rows
        val = int(sum(row[1].values))
        df.loc[index, col_name] = val
        index = index+1
    return df

In [83]:
# for x in range(0, len(df.head(10))):
#     print df.iloc(x)
# for row in df[['num_components', 'num_blocks']].head(1).iterrows():
#     print int(sum(row[1].values))

,id,project_name,user_name,num_screens,b_component_event,b_component_method,b_component_set_get,b_lexical_variable_get,b_logic_false,b_math_add,...,b_math_convert_angles,b_lists_to_csv_table,b_math_trig,b_math_ceiling,b_math_tan,b_lists_to_csv_row,b_lists_append_list,b_math_random_set_seed,c_ProximitySensor,b_obsufcated_text
0,4503600801775616,spaceinvaders,izzydafindon,1,7,1,19,1,4,2,...,0,0,0,0,0,0,0,0,0,0
1,4503601506418688,draw,7724,1,1,1,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4503601917460480,flaggenerator18,scgs111064,1,6,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4503602572820480,paintpot_random,808JRyan92,1,7,3,8,8,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4503603176800256,pongkeyur,keyurpanchal1208,1,6,4,15,7,1,1,...,0,0,0,0,0,0,0,0,0,0
5,4503603769245696,image,h00227559,1,3,0,7,1,3,0,...,0,0,0,0,0,0,0,0,0,0
6,4503605315895296,app3t,yiher_chang,1,4,2,4,6,0,0,...,0,0,0,0,0,0,0,0,0,0
7,4503606143221760,zombielocatoe,northbest71,1,6,6,20,5,2,0,...,0,0,0,0,0,0,0,0,0,0
8,4503606162096128,helloworld1,20127681,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4503607765368832,spaceship,Nuhauskis,1,7,1,19,1,3,2,...,0,0,0,0,0,0,0,0,0,0


0.12031369548584545

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
...
5213    0
5214    0
5215    0
5216    0
5217    0
5218    0
5219    0
5220    0
5221    0
5222    0
5223    0
5224    0
5225    0
5226    0
5227    0
Name: dummy, Length: 5228, dtype: int64